# PESTPP-GLM Part 1

In this notebook, we will run PESTPP-GLM to generate a jco matrix and stop - this is to support data worth testing

In [1]:
%matplotlib inline
import os
import shutil
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib as mpl
plt.rcParams['font.size']=12
import flopy
import pyemu


flopy is installed in /Users/jeremyw/Dev/gw1876/activities_csiro/notebooks/flopy


## SUPER IMPORTANT: SET HOW MANY PARALLEL WORKERS TO USE

In [2]:
num_workers = 20

In [3]:
t_d = "template"
m_d = "master_glm"

In [4]:
pst = pyemu.Pst(os.path.join(t_d,"freyberg.pst"))
pst.write_par_summary_table(filename="none")

,type,transform,count,initial value,upper bound,lower bound,standard deviation
pp_prsity2,pp_prsity2,log,32,0,0.176091,-0.30103,0.11928
gr_sy3,gr_sy3,log,705,0,0.243038,-0.60206,0.211275
pp_rech0,pp_rech0,log,32,0,0.0413927,-0.0457575,0.0217875
cn_hk6,cn_hk6,log,1,0,1,-1,0.5
cn_sy7,cn_sy7,log,1,0,0.243038,-0.60206,0.211275
gr_ss4,gr_ss4,log,705,0,1,-1,0.5
cn_sy6,cn_sy6,log,1,0,0.243038,-0.60206,0.211275
cn_hk8,cn_hk8,log,1,0,1,-1,0.5
pp_vka1,pp_vka1,log,32,0,1,-1,0.5
pp_sy0,pp_sy0,log,32,0,0.243038,-0.60206,0.211275


### reduce the number of adjustable parameters

This is the painful part: we cant use 10K+ pars because we cant wait around for that many runs and then the linear algebra of factoring a 10k+ by 10K+ matrix is also difficult.  So that means we need to fix a lot a parameters #frownyface

In [5]:
par = pst.parameter_data

In [6]:
# grid-scale pars
gr_pars = par.loc[par.pargp.apply(lambda x: "gr" in x),"parnme"]
par.loc[gr_pars,"partrans"] = "fixed"
pst.npar_adj

719

In [7]:
par.loc[par.pargp.apply(lambda x: "pp" in x),"pargp"].unique()

array(['pp_hk0', 'pp_hk1', 'pp_hk2', 'pp_prsity0', 'pp_prsity1',
       'pp_prsity2', 'pp_rech0', 'pp_rech1', 'pp_ss0', 'pp_ss1', 'pp_ss2',
       'pp_strt0', 'pp_strt1', 'pp_strt2', 'pp_sy0', 'pp_sy1', 'pp_sy2',
       'pp_vka0', 'pp_vka1', 'pp_vka2'], dtype=object)

Fix the storage pilot points - we still have layer-scale storage pars adjustable

In [8]:
#s_pars = par.loc[par.pargp.apply(lambda x: "pp" in x and ("ss" in x or "sy" in x)),"parnme"]
#par.loc[s_pars,"partrans"] = "fixed"
pst.npar_adj

719

In [9]:
adj_par = par.loc[par.partrans=="log",:]
adj_par.pargp.value_counts().sort_values()

flow            1
cn_vka7         1
cn_strt8        1
cn_hk6          1
cn_sy6          1
cn_hk8          1
cn_prsity7      1
cn_strt7        1
cn_prsity8      1
cn_ss6          1
cn_prsity6      1
cn_vka6         1
cn_rech4        1
cn_hk7          1
cn_vka8         1
cn_strt6        1
cn_sy8          1
cn_ss8          1
cn_rech5        1
cn_ss7          1
cn_sy7          1
welflux         2
welflux_k02     6
drncond_k00    10
pp_prsity2     32
pp_sy1         32
pp_ss0         32
pp_ss2         32
pp_rech0       32
pp_vka2        32
pp_sy0         32
pp_strt0       32
pp_hk0         32
pp_strt1       32
pp_hk2         32
pp_rech1       32
pp_ss1         32
pp_prsity0     32
pp_vka1        32
pp_prsity1     32
pp_sy2         32
pp_vka0        32
pp_hk1         32
pp_strt2       32
strk           40
Name: pargp, dtype: int64

fix the future recharge pilot points, vka in layers 1 and 3 and the initial condition pilot points (we still have layer-scale pars for each of these types)

In [10]:
fi_grps = ["pp_rech1","pp_vka0","pp_vka2","pp_strt0","pp_strt1","pp_strt2"]
par.loc[par.pargp.apply(lambda x: x in fi_grps),"partrans"] = "fixed"
pst.npar_adj

527

Ok, thats better...so lets run PESTPP-GLM.  We will use a single "base parameter" jacobian matrix as the basis for 6 super parameter iterations.  Then we will draw 100 realizations from the FOSM posterior parameter covariance matrix and run those 100 realizations to get the psoterior forecast PDFs

In [11]:
pst.control_data.noptmax = -1
pst.write(os.path.join(t_d,"freyberg_pp.pst"))

noptmax:-1, npar_adj:527, nnz_obs:14


In [12]:
pyemu.os_utils.start_slaves(t_d,"pestpp-glm","freyberg_pp.pst",num_slaves=num_workers,slave_root=".",
                           master_dir=m_d)

That is all we need for FOSM, so stop here and relax!